# Panel Regression - Firm Characteristics

A panel regression is a suitable regression method for analyzing our data, especially since you have quarterly data for multiple companies over a long period of time. A panel regression model accounts for both within-entity and between-entity variations, making it a useful tool to analyze data with a time series and cross-sectional dimension.  
With panel data, we can control for individual-level characteristics that may affect forecast accuracy by including fixed effects for each company or industry. We can also account for time-specific factors that may affect forecast accuracy by including time fixed effects or time-varying covariates.

### Random Effects Panel Regression

In [59]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects


Data

In [60]:
df = pd.read_csv("Dataframes/characteristics.csv")
#X_var_names = ["Revenue - Actual","Enterprise Value","Market Capitalization","loss firm status", "Enterprise Value To Sales (Daily Time Series Ratio)", "3 Month Total Return", "Volume"]
df

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,POOL.OQ,2022-10-01,Retailing,4.196,4.196,1.615339e+09,1.406160e+10,1.259813e+10,2.383926,-9.164375,238539.0,-1
1,POOL.OQ,2022-07-01,Retailing,1.503,1.503,2.055818e+09,1.556187e+10,1.405795e+10,2.749481,-21.661570,185584.0,-1
2,POOL.OQ,2022-04-01,Retailing,34.342,34.342,1.412650e+09,1.843682e+10,1.696712e+10,3.422880,-25.160664,290897.0,-1
3,POOL.OQ,2022-01-01,Retailing,40.267,40.267,1.035557e+09,2.384882e+10,2.268979e+10,4.504382,30.489705,82349.0,-1
4,POOL.OQ,2021-10-01,Retailing,17.194,17.194,1.411448e+09,1.769954e+10,1.742020e+10,3.684400,-5.130322,157740.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
19299,AVY.N,2014-01-01,Materials,1.471,1.471,1.583900e+09,5.553068e+09,4.877168e+09,0.920659,16.003852,157761.0,-1
19300,AVY.N,2013-10-01,Materials,8.095,8.095,1.504900e+09,5.029514e+09,4.273414e+09,0.903176,2.467620,236291.0,-1
19301,AVY.N,2013-07-01,Materials,1.068,1.068,1.552300e+09,5.436229e+09,4.258229e+09,0.906078,-0.054218,0.0,-1
19302,AVY.N,2013-04-01,Materials,2.482,2.482,1.498900e+09,5.459128e+09,4.309428e+09,0.895787,24.148726,0.0,-1


In [61]:
df["Date"] = pd.to_datetime(df["Date"])

Remove outliers

In [62]:
#removing outliers
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_clean = df.copy()
df_clean["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_clean = df_clean.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_clean = df_clean.dropna()
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,POOL.OQ,2022-10-01,Retailing,4.196,4.196,1.615339e+09,1.406160e+10,1.259813e+10,2.383926,-9.164375,238539.0,-1
1,POOL.OQ,2022-07-01,Retailing,1.503,1.503,2.055818e+09,1.556187e+10,1.405795e+10,2.749481,-21.661570,185584.0,-1
2,POOL.OQ,2022-04-01,Retailing,34.342,34.342,1.412650e+09,1.843682e+10,1.696712e+10,3.422880,-25.160664,290897.0,-1
3,POOL.OQ,2022-01-01,Retailing,40.267,40.267,1.035557e+09,2.384882e+10,2.268979e+10,4.504382,30.489705,82349.0,-1
4,POOL.OQ,2021-10-01,Retailing,17.194,17.194,1.411448e+09,1.769954e+10,1.742020e+10,3.684400,-5.130322,157740.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
19299,AVY.N,2014-01-01,Materials,1.471,1.471,1.583900e+09,5.553068e+09,4.877168e+09,0.920659,16.003852,157761.0,-1
19300,AVY.N,2013-10-01,Materials,8.095,8.095,1.504900e+09,5.029514e+09,4.273414e+09,0.903176,2.467620,236291.0,-1
19301,AVY.N,2013-07-01,Materials,1.068,1.068,1.552300e+09,5.436229e+09,4.258229e+09,0.906078,-0.054218,0.0,-1
19302,AVY.N,2013-04-01,Materials,2.482,2.482,1.498900e+09,5.459128e+09,4.309428e+09,0.895787,24.148726,0.0,-1


Standardising

In [63]:
rescale = df_clean
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["Revenue - Actual"] = MinMaxScaler().fit_transform(np.array(rescale["Revenue - Actual"]).reshape(-1,1))
rescale["Enterprise Value"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value"]).reshape(-1,1))
rescale["Market Capitalization"] = MinMaxScaler().fit_transform(np.array(rescale["Market Capitalization"]).reshape(-1,1))
rescale["Enterprise Value To Sales (Daily Time Series Ratio)"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value To Sales (Daily Time Series Ratio)"]).reshape(-1,1))
rescale["3 Month Total Return"] = MinMaxScaler().fit_transform(np.array(rescale["3 Month Total Return"]).reshape(-1,1))
rescale["Volume"] = MinMaxScaler().fit_transform(np.array(rescale["Volume"]).reshape(-1,1))
df_clean = rescale
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,POOL.OQ,2022-10-01,Retailing,4.196,4.196,0.065637,0.031045,0.004337,0.029064,0.235367,0.001361,-1
1,POOL.OQ,2022-07-01,Retailing,1.503,1.503,0.068358,0.031540,0.004842,0.029495,0.195817,0.001059,-1
2,POOL.OQ,2022-04-01,Retailing,34.342,34.342,0.064385,0.032488,0.005848,0.030290,0.184744,0.001660,-1
3,POOL.OQ,2022-01-01,Retailing,40.267,40.267,0.062055,0.034274,0.007826,0.031566,0.360861,0.000470,-1
4,POOL.OQ,2021-10-01,Retailing,17.194,17.194,0.064377,0.032245,0.006004,0.030598,0.248134,0.000900,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
19299,AVY.N,2014-01-01,Materials,1.471,1.471,0.065443,0.028238,0.001667,0.027338,0.315017,0.000900,-1
19300,AVY.N,2013-10-01,Materials,8.095,8.095,0.064955,0.028065,0.001459,0.027317,0.272179,0.001348,-1
19301,AVY.N,2013-07-01,Materials,1.068,1.068,0.065247,0.028199,0.001453,0.027321,0.264198,0.000000,-1
19302,AVY.N,2013-04-01,Materials,2.482,2.482,0.064917,0.028207,0.001471,0.027308,0.340793,0.000000,-1


In [64]:
df_panel = df_clean.copy()
df_panel

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,POOL.OQ,2022-10-01,Retailing,4.196,4.196,0.065637,0.031045,0.004337,0.029064,0.235367,0.001361,-1
1,POOL.OQ,2022-07-01,Retailing,1.503,1.503,0.068358,0.031540,0.004842,0.029495,0.195817,0.001059,-1
2,POOL.OQ,2022-04-01,Retailing,34.342,34.342,0.064385,0.032488,0.005848,0.030290,0.184744,0.001660,-1
3,POOL.OQ,2022-01-01,Retailing,40.267,40.267,0.062055,0.034274,0.007826,0.031566,0.360861,0.000470,-1
4,POOL.OQ,2021-10-01,Retailing,17.194,17.194,0.064377,0.032245,0.006004,0.030598,0.248134,0.000900,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
19299,AVY.N,2014-01-01,Materials,1.471,1.471,0.065443,0.028238,0.001667,0.027338,0.315017,0.000900,-1
19300,AVY.N,2013-10-01,Materials,8.095,8.095,0.064955,0.028065,0.001459,0.027317,0.272179,0.001348,-1
19301,AVY.N,2013-07-01,Materials,1.068,1.068,0.065247,0.028199,0.001453,0.027321,0.264198,0.000000,-1
19302,AVY.N,2013-04-01,Materials,2.482,2.482,0.064917,0.028207,0.001471,0.027308,0.340793,0.000000,-1


#### Fixed Effects

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

In [65]:
df_clean['Instrument'] = df_clean['Instrument'].astype('category')
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(['Instrument', 'Date'], inplace=True)
df_clean

GICS Industry Group Name  \
Instrument Date                                  
POOL.OQ    2022-10-01                Retailing   
           2022-07-01                Retailing   
           2022-04-01                Retailing   
           2022-01-01                Retailing   
           2021-10-01                Retailing   
...                                        ...   
AVY.N      2014-01-01                Materials   
           2013-10-01                Materials   
           2013-07-01                Materials   
           2013-04-01                Materials   
           2013-01-01                Materials   

                       Earnings Per Share - Actual Surprise  \
Instrument Date                                               
POOL.OQ    2022-10-01                                 4.196   
           2022-07-01                                 1.503   
           2022-04-01                                34.342   
           2022-01-01                                40.267   
           2021-10-01                                17.194   
...                                                     ...   
AVY.N      2014-01-01                                 1.471   
           2013-10-01                                 8.095   
           2013-07-01                                 1.068   
           2013-04-01                                 2.482   
           2013-01-01                                11.178   

                       Earnings Per Share - Actual Surprise AbsVals  \
Instrument Date                                                       
POOL.OQ    2022-10-01                                         4.196   
           2022-07-01                                         1.503   
           2022-04-01                                        34.342   
           2022-01-01                                        40.267   
           2021-10-01                                        17.194   
...                                                             ...   
AVY.N      2014-01-01                                         1.471   
           2013-10-01                                         8.095   
           2013-07-01                                         1.068   
           2013-04-01                                         2.482   
           2013-01-01                                        11.178   

                       Revenue - Actual  Enterprise Value  \
Instrument Date                                             
POOL.OQ    2022-10-01          0.065637          0.031045   
           2022-07-01          0.068358          0.031540   
           2022-04-01          0.064385          0.032488   
           2022-01-01          0.062055          0.034274   
           2021-10-01          0.064377          0.032245   
...                                 ...               ...   
AVY.N      2014-01-01          0.065443          0.028238   
           2013-10-01          0.064955          0.028065   
           2013-07-01          0.065247          0.028199   
           2013-04-01          0.064917          0.028207   
           2013-01-01          0.065123          0.027862   

                       Market Capitalization  \
Instrument Date                                
POOL.OQ    2022-10-01               0.004337   
           2022-07-01               0.004842   
           2022-04-01               0.005848   
           2022-01-01               0.007826   
           2021-10-01               0.006004   
...                                      ...   
AVY.N      2014-01-01               0.001667   
           2013-10-01               0.001459   
           2013-07-01               0.001453   
           2013-04-01               0.001471   
           2013-01-01               0.001166   

                       Enterprise Value To Sales (Daily Time Series Ratio)  \
Instrument Date                                                              
POOL.OQ    2022-10-01                               

In [66]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise"]
X = df_clean.loc[:, "Revenue - Actual":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


Dep. Variable:,Earnings Per Share - Actual Surprise,R-squared:,0.0080
Estimator:,PanelOLS,R-squared (Between):,0.5637
No. Observations:,18533,R-squared (Within):,0.0125
Date:,"Tue, Feb 28 2023",R-squared (Overall):,0.1451
Time:,21:48:51,Log-likelihood,-7.746e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,20.832
Entities:,501,P-value,0.0000
Avg Obs:,36.992,Distribution:,"F(7,17985)"
Min Obs:,6.0000,,
Max Obs:,40.000,F-statistic (robust):,20.832


In [67]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_clean.loc[:, "Revenue - Actual":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0070
Estimator:,PanelOLS,R-squared (Between):,-0.3487
No. Observations:,18533,R-squared (Within):,0.0098
Date:,"Tue, Feb 28 2023",R-squared (Overall):,-0.1982
Time:,21:48:51,Log-likelihood,-7.248e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,18.101
Entities:,501,P-value,0.0000
Avg Obs:,36.992,Distribution:,"F(7,17985)"
Min Obs:,6.0000,,
Max Obs:,40.000,F-statistic (robust):,18.101
